In [1]:
import pandas as pd
import os
from pathlib import Path
import gc
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

import cudf
import numba
from numba import cuda
import numpy as np

# parallelize the process on all columns using joblib
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

from src.data_utils import (
    load_recent_data_from_file,
    save_daily_data,
    save_in_folders,
    get_latest_date,
    read_available_dates
)
from src.config import (
    DAILY_DATA_DIR,
    DATA_DIR,
    DAILY_PRIMARY_FEATURES_DIR,
    DAILY_SECONDARY_FEATURES_DIR,
    DAILY_SCALED_FEATURES_DIR,
)


In [2]:
FROM_SCRATCH = True

In [9]:
def apply_cut(df, cols):
    _res = Parallel(
        n_jobs=10,
    )(delayed(pd.qcut)(df[col], q=5, labels=False, duplicates="drop") for col in cols)
    _res = pd.concat(_res, axis=1).astype("int8")
    #print(_res)
    assert abs(_res["feature_2_ratio_rsi_50_close"].mean() - 2) < 0.05, "mean should be 2"
    return _res


def apply_cut_cpu(recent_data, feature_columns):
    recent_data = recent_data.dropna(subset=feature_columns, axis=0)
    recent_data_gpu = cudf.DataFrame.from_pandas(recent_data)

    ranks = (
        recent_data_gpu[["date"] + feature_columns]
        .groupby("date")
        .rank(pct=True, method="first", ascending=True, na_option="keep")
    )
    ranks["date"] = recent_data["date"]
    ranks["bloomberg_ticker"] = recent_data["bloomberg_ticker"]

    ranks_pd = ranks.to_pandas()

    del recent_data_gpu, ranks

    res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
    res["date"] = ranks_pd["date"]
    res["bloomberg_ticker"] = ranks_pd["bloomberg_ticker"]

    return res


In [10]:
dates = read_available_dates(DAILY_SECONDARY_FEATURES_DIR)

start_index = len(dates) - 1000 if not FROM_SCRATCH else 0

# iterate over all dates in chunks of 200
for i in tqdm(range(start_index, len(dates), 200)):
    print(i)
    _tmp = load_recent_data_from_file(
        DAILY_SECONDARY_FEATURES_DIR,
        n_days=200,
        ascending=True,
        offset=i,
        dtype="float32",
    )
    # print max and min date
    print(_tmp["date"].min(), _tmp["date"].max())
    _tmp = _tmp.reset_index(drop=True)
    _tmp = _tmp.sort_values(["date", "bloomberg_ticker"])
    _tmp = _tmp.groupby("date").filter(lambda x: len(x) > 10)

    # get all feature columns
    feature_columns = [f for f in _tmp.columns if f.startswith("feature_")]

    # apply cut
    res = apply_cut_cpu(_tmp, feature_columns)

    # save
    save_in_folders(res, DAILY_SCALED_FEATURES_DIR)

    #del _tmp, res
    gc.collect()
    

  0%|          | 0/31 [00:00<?, ?it/s]

0
2000-01-04 00:00:00 2000-10-09 00:00:00


/tmp/ipykernel_12372/3885321339.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
  3%|▎         | 1/31 [00:15<07:38, 15.28s/it]

200
2000-10-10 00:00:00 2001-07-18 00:00:00


/tmp/ipykernel_12372/3885321339.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
  6%|▋         | 2/31 [00:38<09:46, 20.23s/it]

...

/tmp/ipykernel_12372/3885321339.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
 94%|█████████▎| 29/31 [13:01<01:01, 30.50s/it]

5800
2022-04-21 00:00:00 2023-01-25 00:00:00


/tmp/ipykernel_12372/3885321339.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
 97%|█████████▋| 30/31 [13:31<00:30, 30.33s/it]

6000
2023-01-26 00:00:00 2023-07-07 00:00:00


/tmp/ipykernel_12372/3885321339.py:27: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  res = ranks_pd.groupby("date").apply(lambda df: apply_cut(df, feature_columns))
100%|██████████| 31/31 [13:49<00:00, 26.77s/it]
